In [1]:
import pandas as pd
import numpy as np
import oci
import getpass
import oracledb
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
# #Replace the dsn,config_dir,wallet_location,wallet_password below with your values.
# pw = getpass.getpass("Enter password:")
# wallet_pw=getpass.getpass("Enter wallet password:")
# connection=oracledb.connect(
#      user="admin",
#      password="Password of your database you set during creation",
#      dsn="Connection string in tnsnames.ora file",
#      config_dir= "Location of the tnsnames.ora file",
#      wallet_location="Location where the wallet is downloaded",
#      wallet_password="Password of the wallet fiel downloaded when creating the Autonomoud database")

In [ ]:
connection=oracledb.connect(
    #By default the username is admin when you create the database
     user=os.getenv("USERNAME"),
    #Password of your database you set during creation
     password=os.getenv("PASSWORD"), 
    #Connection string in tnsnames.ora file
     dsn=os.getenv("DSN"),
    #Location of the tnsnames.ora file
     config_dir=os.getenv("CONFIG_DIR"),
    #Location where the wallet is downloaded
     wallet_location=os.getenv("WALLET_LOCATION"),
    #Password of the wallet fiel downloaded when creating the Autonomoud database
     wallet_password=os.getenv("WALLET_PASSWORD"))

In [2]:
data=pd.read_csv("custom_pred.csv")

In [ ]:
data.rename(columns={'labels': 'model_pred'}, inplace=True)

In [ ]:
data.rename(columns={'sentiment': 'actual_label'}, inplace=True)

In [ ]:
rows2 = list(data.to_records(index=False))

In [ ]:
for i in range(len(rows2)):
        rows2[i] = list(rows2[i])  # Convert tuple to list
        rows2[i][-4] = np.int64(rows2[i][-4]).item()
        rows2[i][-3] = np.float64(rows2[i][-3]).item()
        rows2[i][-2] = np.int64(rows2[i][-2]).item()
        rows2[i] = list(rows2[i])

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("""
        begin
            execute immediate 'drop table twitter_sentiments';
            exception when others then if sqlcode <> -942 then raise; end if;
        end;""")
    cursor.execute("""
        CREATE TABLE twitter_sentiments (
            textID VARCHAR2(4000),
            text VARCHAR2(4000),
            selected_text VARCHAR2(4000),
            actual_label VARCHAR2(4000),
            "time_of_tweet" VARCHAR2(4000),
            "age_of_user" VARCHAR2(4000),
            country VARCHAR2(4000),
            "population" INTEGER ,
            "land_area" FLOAT(10),
            "density" INTEGER ,
             model_pred VARCHAR2(4000)
        )\
        """)
    data = rows2

    sql = """
        INSERT INTO twitter_sentiments (textID, text, selected_text, actual_label, "time_of_tweet", "age_of_user", country, "population", "land_area", "density", model_pred)
        VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11)
        """

    for row in data:
        cursor.execute(sql, row)

# Commit the changes
connection.commit()
